In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import time

In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time

# Load the CSV file containing URLs
data = pd.read_csv('../data/sideshow/sideshow_2018_2019.csv') 
urls = data['url'].tolist() 

# Function to fetch the main text content from a webpage
def fetch_text_from_url(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Check if the request was successful
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract content within <p> tags
        paragraphs = soup.find_all('p')
        text_content = "\n".join([p.get_text() for p in paragraphs if p.get_text()])
        
        return text_content
    except requests.exceptions.HTTPError as http_err:
        print(f"Cannot fetch {url}, HTTP error: {http_err}")
    except requests.exceptions.Timeout:
        print(f"Cannot fetch {url}, connection timed out.")
    except requests.exceptions.RequestException as e:
        print(f"Cannot fetch {url}, error: {e}")
    return None

# Scrape text content for each URL and display progress
all_texts = []
for url in tqdm(urls, desc="Scraping Progress", leave=True):  # `leave=True` keeps progress bar on one line
    text = fetch_text_from_url(url)
    if text:
        print(f"Successfully fetched: {url}")  # Print success message for each URL fetched
        all_texts.append(text)
    else:
        all_texts.append("")  # Add an empty string if fetching fails
    time.sleep(1)  # Add delay to avoid being blocked

# Add the fetched content to the DataFrame and save to a new CSV file
data['text'] = all_texts
data.to_csv('/mnt/data/sideshow_with_texts.csv', index=False)
print("Scraping completed, file saved as sideshow_with_texts.csv")


Scraping Progress:   0%|          | 0/8402 [00:00<?, ?it/s]

Successfully fetched: https://thetakeout.com/with-gwen-s-peanut-butter-better-sauce-even-broccoli-i-1826413956


Scraping Progress:   0%|          | 1/8402 [00:01<3:01:09,  1.29s/it]

Successfully fetched: https://gizmodo.com/after-trump-rants-top-gop-senator-asks-ftc-to-re-open-1828724049


Scraping Progress:   0%|          | 2/8402 [00:02<2:49:21,  1.21s/it]

...

Scraping Progress:   2%|▏         | 136/8402 [03:38<3:23:32,  1.48s/it]

Successfully fetched: https://www.yahoo.com/news/senator-reaches-nonexistent-glasses-takes-223156000.html


Scraping Progress:   2%|▏         | 137/8402 [03:40<3:29:34,  1.52s/it]

Cannot fetch https://www.marketwatch.com/story/moderator-who-botched-pennsylvanias-single-gubernatorial-debate-who-is-alex-trebek-2018-10-04, HTTP error: 401 Client Error: HTTP Forbidden for url: https://www.marketwatch.com/story/moderator-who-botched-pennsylvanias-single-gubernatorial-debate-who-is-alex-trebek-2018-10-04


Scraping Progress:   2%|▏         | 138/8402 [03:41<3:13:10,  1.40s/it]